In [1]:
import onnx

In [2]:
! ls model.onnx

model.onnx


In [3]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.2


In [4]:
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig

spAuth = ServicePrincipalAuthentication(
    tenant_id="TENANT_ID",
    username="SERVICE_PRINCIPAL_ID", 
    password="SERVICE_PRINCIPAL_PASSWORD")

subscription_id = "SUBSCRIPTION_ID"
resource_group = "RESOURCE_GROUP_NAME"
workspace_name = "WORKSPACE_NAME"

ws = Workspace(auth=spAuth, subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
print("Loaded workspace: " + ws.name)

Loaded workspace: WORKSPACE_NAME


In [5]:
from azureml.core.model import Model

model = Model.register(model_path = "./model.onnx",
                       model_name = "tiny_yolov2",
                       tags = {"onnx": "demo"},
                       description = "Tiny YOLO model",
                       workspace = ws)

Registering model tiny_yolov2


In [6]:
%%writefile score.py
import json
import numpy as np
import onnxruntime
import sys
import os
from azureml.core.model import Model
import time
from json_tricks import dumps

def init():
    global session, input_name, output_name
        
    model = Model.get_model_path(model_name = 'tiny_yolov2')
    session = onnxruntime.InferenceSession(model, None)
    
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name 
    
def run(input_data):
    try:
        # load in our data, convert to readable format
        data = np.array(json.loads(input_data)['data']).astype('float32')

        r = session.run([output_name], {input_name: data})[0]

        serialised = dumps(r)
        result_dict = {"result": serialised}
        
    except Exception as e:
        result_dict = json.dumps({"error": str(e)})
    return result_dict

Writing score.py


In [7]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("onnxruntime")
myenv.add_pip_package("json-tricks")
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [8]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  description = "A container image to host YOLO model",
                                                  tags = {"demo": "onnx"})
image = ContainerImage.create(name = "yoloimage",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running...................
SucceededImage creation operation finished for image yoloimage:11, operation "Succeeded"


In [9]:
print(image.image_build_log_uri)

https://eastus2ice.blob.core.windows.net/logs/WORKSPACE_NAME3353554584_e5040f62703947db929516aef27f56c9.txt?se=2019-01-14T05%3A49%3A06Z&sv=2017-04-17&sig=piiYUSFR1PQ/w3WQ%2BUaQereNdl86Msi8S2nP3QUn7A4%3D&sp=r&sr=b


In [10]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'ONNX for yolo model')

In [11]:
from azureml.core.webservice import Webservice

aci_service_name = 'onnx-demo-yolo'
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)

print(aci_service.state)

Creating service
Running.........
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [12]:
print(aci_service.scoring_uri)

http://13.68.226.236:80/score
